# Introduction

In [1]:
from ipywebcam import WebCamWidget, WebCamRecorder, Record, RecordPlayer, FileListFactory

In [2]:
factory = FileListFactory("test", "$i4-$Y-$m-$d-$H-${M}-${S}-$f-$uh6.mp4", condition=FileListFactory.create_frame_based_condition(100))

In [6]:
import random
def add_random_stats(frame, record: Record, ctx):
    record.set_statistics('test1', frame.time, random.randint(0, 30), flush=False)
    record.set_statistics('test2', frame.time, random.randint(0, 15), flush=False)
    record.set_statistics('test3', frame.time, random.randint(-10, 10), flush=False)
transformer = Record.create_video_frame_transformer(add_random_stats)

factory.load()
for r in factory:
    r.read([transformer])
    r.set_statistics_meta_item('test1', 'y_range', [0, 30])
    r.set_statistics_meta_item('test2', 'y_range', [-5, 20])
    r.set_statistics_meta_item('test3', 'y_range', [-5, 20])
factory.save()

load 59 records


Exception: This is a exception

In [3]:
w = WebCamWidget(
    iceServers=[{"urls": "turn:190.92.221.222", "username": "admin", "credential": "123456"}],
    constraints={
        "video": {
            "frameRate": { "max": 10 },
            "width": 256,
            "height": 192,
        },
    },
)
w

<super: ipywebcam.webcam.WebCamWidget,
        WebCamWidget(constraints={'video': {'frameRate': {'max': 10}, 'width': 256, 'height': 192}}, iceServers=[{'urls': 'turn:190.92.221.222', 'username': 'admin', 'credential': '123456'}])>

Output()

In [4]:
recorder = WebCamRecorder(w, factory)
recorder.start()

In [6]:
import cv2
import sys
from ipywebcam import ContextHelper
from av import VideoFrame
array = []
async def transform(frame, ctx):
    async with ContextHelper(ctx) as helper:
        if helper.is_first_time_meet():
            print("first time")
        if helper.is_first_time_meet():
            print("first time")
        if helper.is_frame_passed(70):
            print("70 frame has passed")
        if helper.is_frame_passed(70):
            print("70 frame has passed")
        if helper.is_time_passed(10):
            print("10 second has passed")
        if helper.is_time_passed(10):
            print("10 second has passed")
        img = frame.to_ndarray(format="bgr24")
        # prepare color
        img_color = cv2.pyrDown(cv2.pyrDown(img))
        for _ in range(6):
            img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
        img_color = cv2.pyrUp(cv2.pyrUp(img_color))

        # prepare edges
        img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img_edges = cv2.adaptiveThreshold(
            cv2.medianBlur(img_edges, 7),
            255,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            9,
            2,
        )
        img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

        # combine color and edges
        img = cv2.bitwise_and(img_color, img_edges)

        # rebuild a VideoFrame, preserving timing information
        new_frame = VideoFrame.from_ndarray(img, format="bgr24")
        # new_frame.pts = frame.pts
        # new_frame.time_base = frame.time_base
        return new_frame

transformer = w.add_video_transformer(transform)

In [7]:
w.remove_video_transformer(transformer)

In [6]:
w.output.clear_output()

In [7]:
recorder.stop()

In [8]:
w.close_peers()

In [3]:
import logging
import cv2
from av import VideoFrame
from ipywebcam import ContextHelper
logger = logging.getLogger('ipywebcam')
recorder = WebCamRecorder(None, factory)
recorder.factory.load()
print(recorder.factory.record_count())
player = RecordPlayer(recorder=recorder)
player.loop = True
def create_transform(channel):
    def transform(frame, r, ctx):
        with ContextHelper(ctx) as helper:
            if helper.is_frame_passed(30):
                print(id(channel))
            img = frame.to_ndarray(format="bgr24")
            # prepare color
            img_color = cv2.pyrDown(cv2.pyrDown(img))
            for _ in range(6):
                img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
            img_color = cv2.pyrUp(cv2.pyrUp(img_color))

            # prepare edges
            img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img_edges = cv2.adaptiveThreshold(
                cv2.medianBlur(img_edges, 7),
                255,
                cv2.ADAPTIVE_THRESH_MEAN_C,
                cv2.THRESH_BINARY,
                9,
                2,
            )
            img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

            # combine color and edges
            img = cv2.bitwise_and(img_color, img_edges)

            # rebuild a VideoFrame, preserving timing information
            new_frame = VideoFrame.from_ndarray(img, format="bgr24")
            # new_frame.pts = frame.pts
            # new_frame.time_base = frame.time_base
            return new_frame
    return transform
transformers = {}
for channel in ["test1", "test2", "test3"]:
    transform = create_transform(channel)
    transformers[channel] = player.add_video_transformer(transform, channel)

load 59 records
59


In [28]:
for channel, transformer in transformers.items():
    player.remove_transformer(transformer, channel)

In [33]:
player.output.clear_output()

In [4]:
player

<super: ipywebcam.recorder.RecordPlayer,
        RecordPlayer(loop=True, selected_range=[0.0, 0.0])>

Output()

In [11]:
player.output

Output(outputs=({'traceback': ['\x1b--------------------------------------------------------------------…

In [6]:
player.selected_range

[4.0, 7.0]

In [8]:
a = { "a":1, "b":2}
print(a)
del a["b"]
print(a)

{'a': 1, 'b': 2}
{'a': 1}


In [11]:
from os import path
print(path.normpath('\\a\\b\\c'))
print(path.normpath('/a/b/c'))
p = path.relpath(path.normpath('a/b/c'), path.normpath('a/d'))
print(p)

\a\b\c
\a\b\c
..\b\c


In [8]:
import asyncio
async def some_async_function():
    for i in range(0, 10):
        print(i)
loop = asyncio.get_event_loop()
loop.create_task(some_async_function())

<Task pending name='Task-32730' coro=<some_async_function() running at C:\Users\cxj\AppData\Local\Temp\ipykernel_11156\2957130310.py:2>>

0
1
2
3
4
5
6
7
8
9
